In [15]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

In [23]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{'role':'user', 'content': prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0.0, 
    )

    return response.choices[0].message['content']

In [24]:
import json
import os

# get the courses for the current quarter
with open("data/2022.json") as f:
    courses = json.load(f)
courses

{'1b94c60fab8564f8dc0a79b9652378a4b59aaf20621f437e54fbca3850b306c7c47b6c7d740f654708d7146750196468': {'Id': '1b94c60fab8564f8dc0a79b9652378a4b59aaf20621f437e54fbca3850b306c7c47b6c7d740f654708d7146750196468',
  'Report Title': 'HMRT 23450 1, CMSC 10450 1, MAAD 13450 1 - Artificial Intelligence, Algorithms and Human Rights - Instructor(s): Austin Clyde',
  'Catalog Number': [{'Subject': 'HMRT', 'CourseId': '23450', 'Section': '1'},
   {'Subject': 'CMSC', 'CourseId': '10450', 'Section': '1'},
   {'Subject': 'MAAD', 'CourseId': '13450', 'Section': '1'}],
  'Course Name': 'Artificial Intelligence, Algorithms and Human Rights',
  'Instructors': [': Austin Clyde'],
  'Academic Year': 'Autumn',
  'Academic Term': '2022',
  'Number Enrolled': '27',
  'Number Responses': '12',
  'Evaluation Text Data': {'What are the most important things that you learned in this course? Please reflect on the knowledge and skills you gained.': ['How to think about human right law and practices in context with di

In [56]:
from q_type import question_need
import tiktoken 

def num_tokens_from_string(string, encoding_name = 'gpt-3.5-turbo') :
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def construct_prompt(course):
    instruction = "Here are some questions, feedbacks and summaries about the course {}. Combine the summaries \
in three paragraphs. The first paragraph should be about the grading,\
the second paragraph should be about teaching style and any redflags in teaching, and third paragraph\
should be about grading \n".format(course["Course Name"])
    
    comment_sum = "summarize the following feedbacks, especially summarize any red flags about the course"
    text = comment_sum
    summary = ""
    for question, comments in course["Evaluation Text Data"].items():
        if "instructor" not in question and \
            "difficulty" not in question and \
            "elaborate" not in question and \
            "comments" not in question:
            continue
        
        new_text = text + 'Question:' + question + '\n' + 'Feedbacks:\n'

        for i, comment in enumerate(comments):
            new_text += str(i) + '.' + comment + '\n'
        
        if (num_tokens_from_string(new_text) > 3000):
            summary += get_completion(text) + '\n'
            text = comment_sum
        else:
            text = new_text

    summary += get_completion(text) + '\n'

    return instruction + summary

for id, course in courses.items():
    prompt = construct_prompt(course)
    course.pop("Evaluation Text Data")
    course["Summary"] = get_completion(prompt)
    
with open("data/2022-summary.json") as f:
    json.dump(courses, f)
